Last Update @ 2020.12.04

- Huggingface Transformers 4.0.0  버전 반영

# Package 설치 & 데이터 받기

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
!pip install -q transformers pytorch_lightning emoji soynlp

     |████████████████████████████████| 2.6 MB 30.0 MB/s 
     |████████████████████████████████| 813 kB 48.9 MB/s 
     |████████████████████████████████| 185 kB 61.4 MB/s 
     |████████████████████████████████| 416 kB 41.2 MB/s 
     |████████████████████████████████| 3.3 MB 42.6 MB/s 
     |████████████████████████████████| 895 kB 49.7 MB/s 
     |████████████████████████████████| 636 kB 40.5 MB/s 
     |████████████████████████████████| 10.6 MB 43.5 MB/s 
     |████████████████████████████████| 234 kB 59.6 MB/s 
     |████████████████████████████████| 118 kB 60.6 MB/s 
     |████████████████████████████████| 829 kB 51.9 MB/s 
     |████████████████████████████████| 1.3 MB 57.4 MB/s 
     |████████████████████████████████| 294 kB 46.3 MB/s 
     |████████████████████████████████| 142 kB 62.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2

In [ ]:
!git clone https://github.com/e9t/nsmc 

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 21.09 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Checking out files: 100% (14737/14737), done.


In [ ]:
!head nsmc/ratings_train.txt

id	document	label
9976970	아 더빙.. 진짜 짜증나네요 목소리	0
3819312	흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나	1
10265843	너무재밓었다그래서보는것을추천한다	0
9045019	교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정	0
6483659	사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다	1
5403919	막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.	0
7797314	원작의 긴장감을 제대로 살려내지못했다.	0
9443947	별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네	0
7156791	액션이 없는데도 재미 있는 몇안되는 영화	1


# 패키지 import & 기본 Args 설정

In [ ]:
import pandas as pd
import numpy as np
import os
from pprint import pprint

import pickle
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.optim.lr_scheduler import ExponentialLR

from torch import nn
from torch.nn import CrossEntropyLoss, MSELoss
import torch.nn.functional as F

from pytorch_lightning import LightningModule, Trainer, seed_everything
from transformers import BertModel, BertTokenizer, AdamW, BertForSequenceClassification
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from pytorch_lightning.loggers import TensorBoardLogger
from pathlib import Path
from collections import Counter
import random

import re
import emoji
from soynlp.normalizer import repeat_normalize

## 기본 학습 Arguments

In [ ]:
class Arg:
    random_seed: int = 42  # Random Seed
    pretrained_model: str = 'beomi/kcbert-base'  # Transformers PLM name
    pretrained_tokenizer: str = ''  # Optional, Transformers Tokenizer Name. Overrides `pretrained_model`
    auto_batch_size: str = 'power'  # Let PyTorch Lightening find the best batch size 
    batch_size: int = 8  # Optional, Train/Eval Batch Size. Overrides `auto_batch_size` 
    lr: float = 5e-5  # Starting Learning Rate
    epochs: int = 20  # Max Epochs
    max_length: int = 150  # Max Length input size
    report_cycle: int = 100  # Report (Train Metrics) Cycle
    train_data_path: str = "/content/train.csv"  # Train Dataset file
    val_data_path: str = "/content/valid.csv"  # Validation Dataset file
    cpu_workers: int = os.cpu_count()  # Multi cpu workers
    test_mode: bool = False  # Test Mode enables `fast_dev_run`
    optimizer: str = 'AdamW'  # AdamW vs AdamP
    lr_scheduler: str = 'exp'  # ExponentialLR vs CosineAnnealingWarmRestarts
    fp16: bool = False  # Enable train on FP16
    tpu_cores: int = 0  # Enable TPU with 1 core or 8 cores
    hidden_dropout_prob = 0.1 # BERT paper setting 
    hidden_size =  768 # BERT-base: 768, BERT-large: 1024, BERT paper setting
    log_dir = './models/checkpoints'
    log_name = 'kcbert-base-stressor-binary-clf'
    version = 1
    report_cycle: int = 30  # Report (Train Metrics) Cycle


args = Arg()

## 기본값을 Override 하고싶은 경우 아래와 같이 수정

In [ ]:
!nvidia-smi

Tue Jul 27 10:07:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

위에서 GPU가 V100/P100이면 아래 `batch_size`  를 32 이상으로 하셔도 됩니다.

In [ ]:
# args.tpu_cores = 8  # Enables TPU
args.fp16 = True  # Enables GPU FP16
args.batch_size = 16 # Force setup batch_size

# Model 만들기 with Pytorch Lightning

In [ ]:
class Model(LightningModule):
    def __init__(self, options):
        super().__init__()
        # config:
        self.args = options
        # self.bert = BertForSequenceClassification.from_pretrained(self.args.pretrained_model)

        # meta data:
        self.epochs_index = 0
        self.num_labels = 44

        # modules:
        self.tokenizer = BertTokenizer.from_pretrained(
            self.args.pretrained_tokenizer
            if self.args.pretrained_tokenizer
            else self.args.pretrained_model
        )
        self.bert_data = BertModel.from_pretrained(self.args.pretrained_model)
        self.fc1 = nn.Linear(self.args.hidden_size, self.args.hidden_size)
        self.dropout = nn.Dropout(self.args.hidden_dropout_prob)
        self.classifier = nn.Linear(self.args.hidden_size, self.num_labels)

    def forward(self, data, **kwargs):
        # import pdb;pdb.set_trace()
        outputs_data = self.bert_data(data, **kwargs) # return: last_hidden_state, pooler_output, hidden_states, attentions
        output = outputs_data[1]

        pooled_output = self.dropout(self.fc1(output))
        logits = self.classifier(pooled_output)
        return logits

    def step(self, batch, batch_idx):
        # import pdb;pdb.set_trace()
        data, labels = batch
        logits = self(data=data)

        # Transformers 4.0.0+
        loss = None
        loss_fct = CrossEntropyLoss()
        # import pdb;pdb.set_trace()
        loss = loss_fct(logits, labels.view(-1))


        preds = logits.argmax(dim=-1)
        y_true = list(labels.cpu().numpy())
        y_pred = list(preds.cpu().numpy())

        return {
            'loss': loss,
            'y_true': y_true,
            'y_pred': y_pred,
        }
        
    def training_step(self, batch, batch_idx):
        return self.step(batch, batch_idx)

    def validation_step(self, batch, batch_idx):
        return self.step(batch, batch_idx)


    def epoch_end(self, outputs, state='train'):
        
        loss = torch.tensor(0, dtype=torch.float)
        for i in outputs:
            loss += i['loss'].cpu().detach()
        loss = loss / len(outputs)

        y_true = []
        y_pred = []
        for i in outputs:
            y_true += i['y_true']
            y_pred += i['y_pred']
        
        acc = accuracy_score(y_true, y_pred)
        prec = precision_score(y_true, y_pred, average = 'micro')
        rec = recall_score(y_true, y_pred, average = 'micro')
        f1 = f1_score(y_true, y_pred, average = 'micro')

        self.log(state+'_loss', float(loss), on_epoch=True, prog_bar=True)
        self.log(state+'_acc', acc, on_epoch=True, prog_bar=True)
        self.log(state+'_precision', prec, on_epoch=True, prog_bar=True)
        self.log(state+'_recall', rec, on_epoch=True, prog_bar=True)
        self.log(state+'_f1', f1, on_epoch=True, prog_bar=True)
        print(f'[Epoch {self.trainer.current_epoch} {state.upper()}] Loss: {loss}, Acc: {acc}, Prec: {prec}, Rec: {rec}, F1: {f1}')
        return {'loss': loss}
    
    def training_epoch_end(self, outputs):
        self.epoch_end(outputs, state='train')

    def validation_epoch_end(self, outputs):
        self.epoch_end(outputs, state='val')


    def configure_optimizers(self):
        if self.args.optimizer == 'AdamW':
            optimizer = AdamW(self.parameters(), lr=self.args.lr)
        elif self.args.optimizer == 'AdamP':
            from adamp import AdamP
            optimizer = AdamP(self.parameters(), lr=self.args.lr)
        else:
            raise NotImplementedError('Only AdamW and AdamP is Supported!')
        if self.args.lr_scheduler == 'cos':
            scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=1, T_mult=2)
        elif self.args.lr_scheduler == 'exp':
            scheduler = ExponentialLR(optimizer, gamma=0.5)
        else:
            raise NotImplementedError('Only cos and exp lr scheduler is Supported!')
        return {
            'optimizer': optimizer,
            'scheduler': scheduler,
        }

    def read_data(self, path):
        if path.endswith('xlsx'):
            return pd.read_excel(path)
        elif path.endswith('csv'):
            return pd.read_csv(path,header=None,names=['document', 'label'])
        elif path.endswith('tsv') or path.endswith('txt'):
            return pd.read_csv(path, sep='\t')
        else:
            raise NotImplementedError('Only Excel(xlsx)/Csv/Tsv(txt) are Supported')
  
    def clean(self, x):
        emojis = ''.join(emoji.UNICODE_EMOJI.keys())
        pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-힣{emojis}]+')
        url_pattern = re.compile(
            r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')
        x = pattern.sub(' ', x)
        x = url_pattern.sub('', x)
        x = x.strip()
        x = repeat_normalize(x, num_repeats=2)
        return x

    def encode(self, x, **kwargs):
        return self.tokenizer.encode(
            self.clean(str(x)),
            padding='max_length',
            max_length=self.args.max_length,
            truncation=True,
            **kwargs,
        )

    def preprocess_dataframe(self, df):
        df['document'] = df['document'].map(self.encode)
        return df

    def dataloader(self, path, shuffle=False):
        df = self.read_data(path)
        df = self.preprocess_dataframe(df)

        dataset = TensorDataset(
            torch.tensor(df['document'].to_list(), dtype=torch.long),
            torch.tensor(df['label'].to_list(), dtype=torch.long),
        )
        return DataLoader(
            dataset,
            batch_size=self.args.batch_size * 1 if not self.args.tpu_cores else self.args.tpu_cores,
            shuffle=shuffle,
            num_workers=self.args.cpu_workers,
        )

    def train_dataloader(self):
        return self.dataloader(self.args.train_data_path, shuffle=True)

    def val_dataloader(self):
        return self.dataloader(self.args.val_data_path, shuffle=False)


In [ ]:
def main():
    args = Arg()
    print("Using PyTorch Ver", torch.__version__)
    print("Fix Seed:", args.random_seed)
    seed_everything(args.random_seed)
    model = Model(args)
    print(":: Start Training ::")
    trainer = Trainer(
        max_epochs=args.epochs,
        fast_dev_run=args.test_mode,
        num_sanity_val_steps=None if args.test_mode else 0,
        deterministic=True, # ensure full reproducibility from run to run you need to set seeds for pseudo-random generators,
        # For GPU Setup
        gpus=[0] if torch.cuda.is_available() else None,
        precision=16 if args.fp16 else 32
    )
    trainer.fit(model)

# 학습!

> 주의: 1epoch별로 GPU-P100기준 약 1-2시간, GPU V100기준 ~30분이 걸립니다.

> Update @ 2020.09.01
> 최근 Colab Pro에서 V100이 배정됩니다.

```python
# 1epoch 기준 아래 score가 나옵니다.
{'val_acc': 0.90522,
 'val_f1': 0.9049023739289227,
 'val_loss': 0.23429009318351746,
 'val_precision': 0.9143146796431468,
 'val_recall': 0.8956818813808446}
```

In [ ]:
main()

Global seed set to 42


Using PyTorch Ver 1.9.0+cu102
Fix Seed: 42


Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


:: Start Training ::



  | Name       | Type      | Params
-----------------------------------------
0 | bert_data  | BertModel | 108 M 
1 | fc1        | Linear    | 590 K 
2 | dropout    | Dropout   | 0     
3 | classifier | Linear    | 33.8 K
-----------------------------------------
109 M     Trainable params
0         Non-trainable params
109 M     Total params
438.172   Total estimated model params size (MB)


...

[Epoch 11 VAL] Loss: 0.5051890015602112, Acc: 0.895, Prec: 0.895, Rec: 0.895, F1: 0.895
[Epoch 12 TRAIN] Loss: 0.026551205664873123, Acc: 0.99375, Prec: 0.99375, Rec: 0.99375, F1: 0.99375


[Epoch 12 VAL] Loss: 0.44477829337120056, Acc: 0.9, Prec: 0.9, Rec: 0.9, F1: 0.9
[Epoch 13 TRAIN] Loss: 0.020118383690714836, Acc: 0.9925, Prec: 0.9925, Rec: 0.9925, F1: 0.9925


[Epoch 13 VAL] Loss: 0.44223326444625854, Acc: 0.885, Prec: 0.885, Rec: 0.885, F1: 0.885
[Epoch 14 TRAIN] Loss: 0.016742832958698273, Acc: 0.99625, Prec: 0.99625, Rec: 0.99625, F1: 0.99625


[Epoch 14 VAL] Loss: 0.4046279191970825, Acc: 0.905, Prec: 0.905, Rec: 0.905, F1: 0.905
[Epoch 15 TRAIN] Loss: 0.01427601557224989, Acc: 0.99625, Prec: 0.99625, Rec: 0.99625, F1: 0.99625


[Epoch 15 VAL] Loss: 0.42443692684173584, Acc: 0.905, Prec: 0.905, Rec: 0.905, F1: 0.905
[Epoch 16 TRAIN] Loss: 0.013275276869535446, Acc: 0.99625, Prec: 0.99625, Rec: 0.99625, F1: 0.99625
